<a href="https://colab.research.google.com/github/ovbystrova/hse_compling/blob/main/hw1/hw1_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
%%capture
!pip install pymystem3==0.1.10
!pip install pymorphy2[fast]
!pip install rusenttokenize
!pip install razdel

In [69]:
%%capture
from collections import Counter
import re
import string

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from razdel import tokenize
from rusenttokenize import ru_sent_tokenize

In [3]:
%%capture
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/zhivago.txt -O zhivago.txt

In [85]:
with open('zhivago.txt','r', encoding='utf-8', ) as f:
    text = f.read()

text[:500]

'<author><first-name>Борис</first-name><middle-name>Леонидович</middle-name><last-name>Пастернак</last-name></author>\n<book-title>Доктор Живаго</book-title>\n<annotation><p>«Доктор Живаго» - итоговое произведение Бориса Пастернака, книга всей его жизни. Этот роман принес его автору мировую известность и Нобелевскую премию, присуждение которой обернулось для поэта оголтелой политической травлей, обвинениями в «измене Родине» и в результате стоило ему жизни.</p>\n<p>«Доктор Живаго» - роман, сама ткан'

# Task 1

Отчистите текст от мусора (тэгов, хешей и тп) с помощью регулярных выражений. Постарайтесь убрать весь мусор, но если что-то удалить не получается, то не мучайтесь. Главное, чтобы мусор не проявлялся в результатах следующих заданий.

In [5]:
def clean_text(text):
    text = re.sub('\n', ' ', text)
    text = re.search('<body>.*</body>', text).group(0)
    text = re.sub('<[^>]+>', ' ', text)
    text = re.sub(r'–\xa0', '', text)
    text = re.sub('\s{2,}', ' ', text)
    return text

In [87]:
text = clean_text(text)
text[:500]

' Борис Пастернак Доктор Живаго И ДЫШАТ ПОЧВА И СУДЬБА Спустя два года после завершения романа «Доктор Живаго» Борис Пастернак писал: «Я думаю, несмотря на привычность всего того, что продолжает стоять перед нашими глазами и что мы продолжаем слышать и читать, ничего этого больше нет, это уже прошло и состоялось, огромный, неслыханных сил стоивший период закончился и миновал. Освободилось безмерно большое, покамест пустое и не занятое место для нового и еще не бывалого, для того, что будет угадан'

In [7]:
text[-1000:]

'овеческого пробил. Он в руки грешников себя предаст». И лишь сказал, неведомо откуда Толпа рабов и скопище бродяг, Огни, мечи и впереди – Иуда С предательским лобзаньем на устах. Петр дал мечом отпор головорезам И ухо одному из них отсек. Но слышит: «Спор нельзя решать железом, Вложи свой меч на место, человек. Неужто тьмы крылатых легионов Отец не снарядил бы мне сюда? И, волоска тогда на мне не тронув, Враги рассеялись бы без следа. Но книга жизни подошла к странице, Которая дороже всех святынь. Сейчас должно написанное сбыться, Пускай же сбудется оно. Аминь. Ты видишь, ход веков подобен притче И может загореться на ходу. Во имя страшного ее величья Я в добровольных муках в гроб сойду. Я в гроб сойду и в третий день восстану, И, как сплавляют по реке плоты, Ко мне на суд, как баржи каравана, Столетья поплывут из темноты». Примечания 1 В восторге. (Здесь и далее с французского.) 2 Большой круг! Китайская цепочка! 3 Вальс, пожалуйста! 4 На три счета, на два счета. 5 Шиворот-навыворот.

# Task 2

Приведите очищенный текст к нижнему регистру, удалите все знаки пунктуации, разделите на предложения библиотекой rusenttokenize, токенизируйте библиотекой razdel_tokenize. 
Ответьте на следующие вопросы:

1) есть ли в тексте повторяющиеся корректные предложения? если да то какие? (если находится мусор то вернитесь к заданию 1 и постарайтесь избавиться от него)

2) какой самый частотный токен в тексте длиннее 6 символов?

In [10]:
# Приведение к нижнему регистру
text = text.lower()

# Удаление пунктуации (Оставила только точки, чтобы на предложения в дальнейшем делилось нормально)
punctuation = string.punctuation.replace('.', '') + '—“”‘’'
translator = text.maketrans(punctuation, len(punctuation) * ' ')
text = text.translate(translator)

In [88]:
text[:500]

' Борис Пастернак Доктор Живаго И ДЫШАТ ПОЧВА И СУДЬБА Спустя два года после завершения романа «Доктор Живаго» Борис Пастернак писал: «Я думаю, несмотря на привычность всего того, что продолжает стоять перед нашими глазами и что мы продолжаем слышать и читать, ничего этого больше нет, это уже прошло и состоялось, огромный, неслыханных сил стоивший период закончился и миновал. Освободилось безмерно большое, покамест пустое и не занятое место для нового и еще не бывалого, для того, что будет угадан'

In [12]:
# На предложения
sentences = ru_sent_tokenize(text)

# На токены
tokens = list(tokenize(text))
tokens = [token.text for token in tokens]
sentences[:5], tokens[:5]

(['борис пастернак доктор живаго и дышат почва и судьба спустя два года после завершения романа «доктор живаго» борис пастернак писал  «я думаю  несмотря на привычность всего того  что продолжает стоять перед нашими глазами и что мы продолжаем слышать и читать  ничего этого больше нет  это уже прошло и состоялось  огромный  неслыханных сил стоивший период закончился и миновал.',
  'освободилось безмерно большое  покамест пустое и не занятое место для нового и еще не бывалого  для того  что будет угадано чьей либо гениальной независимостью и свежестью  для того  что внушит и подскажет жизнь новых чисел и дней.',
  'сейчас мукою художников будет не то  признаны ли они и признаны ли будут застаивающейся  запоздалой политической современностью или властью  но неспособность совершенно оторваться от понятий  ставших привычными  забыть навязывающиеся навыки  нарушить непрерывность.',
  'надо понять  что все стало прошлым  что конец виденного и пережитого был уже  а не еще предстоит.',
  'надо

In [13]:
# есть ли в тексте повторяющиеся корректные предложения? если да то какие?
counter_sent = Counter(sentences)
counter_sent.most_common(15)

[('да.', 13),
 ('хорошо.', 8),
 ('знаю.', 5),
 ('я знаю.', 5),
 ('погоди.', 4),
 ('спасибо.', 4),
 ('сеялки.', 4),
 ('молотилки».', 4),
 ('но дело не в этом.', 3),
 ('слушай.', 3),
 ('здравствуйте.', 3),
 ('ладно.', 3),
 ('еще бы.', 3),
 ('свеча горела на столе  свеча горела.', 3),
 ('единственно живое и яркое в вас – это то  что вы жили в одно время со мной и меня знали».',
  2)]

In [14]:
# какой самый частотный токен в тексте длиннее 6 символов?
tokens_filtered = [token for token in tokens if len(token) > 6]
counter_tokens = Counter(tokens_filtered)
counter_tokens.most_common(1)

[('андреевич', 289)]

# Task 3

Сделайте стемминг и найдите по несколько частотных примеров на каждый из видов ошибок:

1) два разных слова ошибочно свелись к одинаковой основе

3) слово не изменилось после стемминга (слово должно быть русским и длиннее 4 символов)

***не делайте это задание полностью вручную, напишите правила, которые отловят потенциальные ошибки и выберите из них***

In [16]:
stemmer = SnowballStemmer('russian')

In [17]:
stemms = [(token, stemmer.stem(token)) for token in tokens]
stemms[:10]

[('борис', 'борис'),
 ('пастернак', 'пастернак'),
 ('доктор', 'доктор'),
 ('живаго', 'живаг'),
 ('и', 'и'),
 ('дышат', 'дышат'),
 ('почва', 'почв'),
 ('и', 'и'),
 ('судьба', 'судьб'),
 ('спустя', 'спуст')]

In [24]:
# два разных слова ошибочно свелись к одинаковой основе
def find_mistakes_identical_stem(stemms):
    stemms_dict = {}
    for token, stemm in stemms:
        if stemm not in stemms_dict:
            stemms_dict[stemm] = {token}
        else:
            stemms_dict[stemm].update({token})
    stemms_dict = {key: value for key, value in stemms_dict.items() if len(value) > 1}
    return stemms_dict

stemms_dict = find_mistakes_identical_stem(stemms)
len(stemms_dict)

7664

Не совсем понятно, как разграничить случаи с однокоренными словами (у них одна основа) и случаи с ошибочным определением основы. 

In [30]:
# Даже тут, у нас два разных слова (мука и муки)(стать и сталь), но то, что они приводятся к одной основе кажется логичным
stemms_dict['мук'], stemms_dict['стал']

({'мук', 'мука', 'мукам', 'муках', 'муки', 'мукой', 'мукою', 'муку'},
 {'стал', 'стала', 'стали', 'стало', 'сталось', 'сталь'})

In [33]:
for key, value in list(stemms_dict.items())[50:70]:
    print(key, ': ', value)

дне :  {'дней', 'дне'}
мук :  {'мукам', 'мук', 'муку', 'мукою', 'мукой', 'муки', 'мука', 'муках'}
художник :  {'художника', 'художников', 'художником', 'художники', 'художник'}
то :  {'то', 'тою', 'той'}
призна :  {'признаюсь', 'признанный', 'признать', 'признано', 'признанным', 'признаться', 'признали', 'признал', 'признаны', 'признанного'}
он :  {'они', 'оной', 'он', 'она', 'оно'}
запоздал :  {'запоздалых', 'запоздалой'}
политическ :  {'политическое', 'политическом', 'политическую', 'политической', 'политических', 'политические', 'политического', 'политический', 'политическим', 'политическая', 'политически'}
современ :  {'современных', 'современному', 'современный', 'современною', 'современностью', 'современного', 'современным', 'современной', 'современные', 'современном'}
ил :  {'иль', 'или'}
власт :  {'власть', 'властям', 'власти', 'властью', 'властей', 'властями'}
неспособн :  {'неспособная', 'неспособным', 'неспособность', 'неспособности', 'неспособный'}
совершен :  {'совершенно'

In [46]:
# слово не изменилось после стемминга (слово должно быть русским и длиннее 4 символов)
def find_mistakes_no_changes(stemms):
    mistakes = set()
    for token, stemm in stemms:
        if token == stemm and len(token) > 10: # Поставила фильтр побольше, чтобы более наглядые примеры были
            mistakes.update({(token, stemm)})
    return mistakes
mistakes_no_changes = find_mistakes_no_changes(stemms)
len(mistakes_no_changes), list(mistakes_no_changes)[:5]

(62,
 [('культпросвет', 'культпросвет'),
  ('миропорядок', 'миропорядок'),
  ('военачальник', 'военачальник'),
  ('обстоятельств', 'обстоятельств'),
  ('казимирович', 'казимирович')])

# Task 4

Проанализируйте список стоп-слов из нлтк (для русского). Какие ещё слова вы бы туда добавили? (5 слов будет достаточно, не забудьте аргументировать ваш выбор)

In [51]:
stopwords_ru = stopwords.words("russian")

print(stopwords_ru)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

В принципе, уже существующий спико стопслов мне кажется достаточным для начала. Тут есть и предлоги, и местоимения, союзы. Т.е. различные слова, смысловой нагрузки особо не несущщие сами по себе.  Дальнейшее добавление стопслов уже зависит от тех данных, над которыми мы работаем. 

In [66]:
'каждый' in stopwords_ru, 'хуже' in stopwords_ru, 'твою' in stopwords_ru, 'менее' in stopwords_ru, 'типа' in stopwords_ru, 'снова' in stopwords_ru

(False, False, False, False, False, False)

Слова для добавления я выбирала по следующему принципу: смотрела на список стопслов, если вижу какое-то слово, например, "лучше", то, как мне кажется, должен быть и его антоним ("хуже") в этом списке. Но его нет (что мне кажется не справедливым, раз "лучше" есть). 

Также я добавила бы в словарь стопслов некоторые слова-паразиты, присущие современному русскому языку (например, "типа"). 

# Task 5

Предобработайте текст двумя способами:

1) лемматизируйте токены с помощью pymorphy2

2) лемматизируйте текст с помощью mystem3 

Ответьте на вопрос:
Что в данном случае лучше для лемматизации mystem или pymorphy?

Важно, чтобы ответы на вопросы были аргументированы (как минимум три аргумента).  Анализируйте результаты с языковой точки зрения. Скорость работы простота интерфейса не являются преимуществами или недостатками в рамках этого задания. Идеальный аргумент - библиотека x неправильно обрабатывает вот такое слово/класс слов. 

In [70]:
mystem = Mystem()
morph = MorphAnalyzer()

In [72]:
# pymorphy2
pm_lemmatized = [morph.parse(token)[0].normal_form for token in tokens]

In [73]:
# mystem
mystem_lemmatized = mystem.lemmatize(text)

Cравним то, как справились библиотеки на начале текста и на конце текста

In [74]:
text[:500]

' борис пастернак доктор живаго и дышат почва и судьба спустя два года после завершения романа «доктор живаго» борис пастернак писал  «я думаю  несмотря на привычность всего того  что продолжает стоять перед нашими глазами и что мы продолжаем слышать и читать  ничего этого больше нет  это уже прошло и состоялось  огромный  неслыханных сил стоивший период закончился и миновал. освободилось безмерно большое  покамест пустое и не занятое место для нового и еще не бывалого  для того  что будет угадан'

In [76]:
print(pm_lemmatized[:100])

['борис', 'пастернак', 'доктор', 'живаго', 'и', 'дышать', 'почва', 'и', 'судьба', 'спустя', 'два', 'год', 'после', 'завершение', 'роман', '«', 'доктор', 'живаго', '»', 'борис', 'пастернак', 'писать', '«', 'я', 'думать', 'несмотря', 'на', 'привычность', 'весь', 'тот', 'что', 'продолжать', 'стоять', 'перед', 'наш', 'глаз', 'и', 'что', 'мы', 'продолжать', 'слышать', 'и', 'читать', 'ничего', 'это', 'большой', 'нет', 'это', 'уже', 'пройти', 'и', 'состояться', 'огромный', 'неслыханный', 'сила', 'стоить', 'период', 'закончиться', 'и', 'миновать', '.', 'освободиться', 'безмерно', 'большой', 'покамест', 'пустой', 'и', 'не', 'занятой', 'место', 'для', 'новый', 'и', 'ещё', 'не', 'бывалый', 'для', 'тот', 'что', 'быть', 'угадать', 'чей', 'либо', 'гениальный', 'независимость', 'и', 'свежесть', 'для', 'тот', 'что', 'внушить', 'и', 'подсказать', 'жизнь', 'новый', 'число', 'и', 'день', '.', 'сейчас']


In [79]:
print(mystem_lemmatized[:200])

[' ', 'борис', ' ', 'пастернак', ' ', 'доктор', ' ', 'живаго', ' ', 'и', ' ', 'дышать', ' ', 'почва', ' ', 'и', ' ', 'судьба', ' ', 'спустя', ' ', 'два', ' ', 'год', ' ', 'после', ' ', 'завершение', ' ', 'роман', ' «', 'доктор', ' ', 'живаго', '» ', 'борис', ' ', 'пастернак', ' ', 'писать', '  «', 'я', ' ', 'думать', '  ', 'несмотря', ' ', 'на', ' ', 'привычность', ' ', 'весь', ' ', 'то', '  ', 'что', ' ', 'продолжать', ' ', 'стоять', ' ', 'перед', ' ', 'наш', ' ', 'глаз', ' ', 'и', ' ', 'что', ' ', 'мы', ' ', 'продолжать', ' ', 'слышать', ' ', 'и', ' ', 'читать', '  ', 'ничто', ' ', 'это', ' ', 'больше', ' ', 'нет', '  ', 'это', ' ', 'уже', ' ', 'проходить', ' ', 'и', ' ', 'состояться', '  ', 'огромный', '  ', 'неслыханный', ' ', 'сила', ' ', 'стоить', ' ', 'период', ' ', 'заканчиваться', ' ', 'и', ' ', 'миновать', '.', ' ', 'освобождаться', ' ', 'безмерно', ' ', 'большой', '  ', 'покамест', ' ', 'пустой', ' ', 'и', ' ', 'не', ' ', 'занимать', ' ', 'место', ' ', 'для', ' ', 'новый', '

In [ ]:
# Как в тесте -- Mystem -- Pymorphy
# ################################
# того -- то -- тот
# прошло -- проходить -- пройти
# освободилось -- освобождаться -- освободиться 
# занятое -- занимать -- занятой
# угадан -- угадать -- угадывать

Можно заметить, что майстем и пайморфи по-разному в некоторых случаях предсказывают лемму. Я бы сказала, что разбор от Майстема подходит в данных случаях больше (особенно в случаях с "занятое", "угадан", "того"), т.к. варианты майстема более полно отражают леммы. Получается, pymorphy может плохо справляться с краткими причастиями. 

In [80]:
text[-500:]

'ь. сейчас должно написанное сбыться  пускай же сбудется оно. аминь. ты видишь  ход веков подобен притче и может загореться на ходу. во имя страшного ее величья я в добровольных муках в гроб сойду. я в гроб сойду и в третий день восстану  и  как сплавляют по реке плоты  ко мне на суд  как баржи каравана  столетья поплывут из темноты». примечания 1 в восторге.  здесь и далее с французского.  2 большой круг  китайская цепочка  3 вальс  пожалуйста  4 на три счета  на два счета. 5 шиворот навыворот. '

In [83]:
print(pm_lemmatized[-40:])

['как', 'баржа', 'караван', 'столетие', 'поплыть', 'из', 'темнота', '»', '.', 'примечание', '1', 'в', 'восторг', '.', 'здесь', 'и', 'далее', 'с', 'французский', '.', '2', 'большой', 'круг', 'китайский', 'цепочка', '3', 'вальс', 'пожалуйста', '4', 'на', 'три', 'счёт', 'на', 'два', 'счёт', '.', '5', 'шиворот', 'навыворот', '.']


In [84]:
print(mystem_lemmatized[-80:])

['ко', ' ', 'я', ' ', 'на', ' ', 'суд', '  ', 'как', ' ', 'баржа', ' ', 'караван', '  ', 'столетие', ' ', 'поплыть', ' ', 'из', ' ', 'темнота', '»', '.', ' ', 'примечание', ' ', '1', ' ', 'в', ' ', 'восторг', '.', '  ', 'здесь', ' ', 'и', ' ', 'далее', ' ', 'с', ' ', 'французский', '.  ', '2', ' ', 'большой', ' ', 'круг', '  ', 'китайский', ' ', 'цепочка', '  ', '3', ' ', 'вальс', '  ', 'пожалуйста', '  ', '4', ' ', 'на', ' ', 'три', ' ', 'счет', '  ', 'на', ' ', 'два', ' ', 'счет', '. ', '5', ' ', 'шиворот', ' ', 'навыворот', '.', ' \n']


В конце текста особых отличий нет между разборами.